### This notebook presents an execution tree enhanced with provenance.
### In this view, the nodes that are not function calls are removed 

In [23]:
# Imports
import sqlite3

from graphviz import Graph

from debugprov.validity import Validity
from debugprov.node import Node
from debugprov.execution_tree import ExecutionTree
from debugprov.execution_tree_creator import ExecTreeCreator
from debugprov.top_down import TopDown
from debugprov.heaviest_first import HeaviestFirst
from debugprov.visualization import Visualization
from debugprov.provenance_enhancement import ProvenanceEnhancement 
from debugprov.single_stepping import SingleStepping
from debugprov.divide_and_query import DivideAndQuery

In [24]:
NOW2_SQLITE_PATH = 'C:/Users/linha/Desktop/ws/py-scripts-examples/text-processing/.noworkflow/db.sqlite'

In [25]:
CURSOR = sqlite3.connect(NOW2_SQLITE_PATH).cursor()

In [26]:
creator = ExecTreeCreator(CURSOR)
exec_tree = creator.create_exec_tree()

In [27]:
prov = ProvenanceEnhancement(exec_tree, CURSOR)

In [28]:
prov.enhance_all()

In [29]:
class CustomVisualization(Visualization):

    # CUSTOM ATTRIBUTES
    PROVENANCE_NODE_COLOR = 'lightblue'
    
    def generate_exec_tree(self, graph_name = 'exec_tree'):
        file_name = "{}.gv".format(graph_name)
        self.graph = Graph(graph_name, filename=file_name)
        self.graph.attr('node', shape='box')
        self.graph.attr('graph', ordering='out')
        root_node = self.exec_tree.root_node
        self.graph.node(str(root_node.ev_id), root_node.get_name(), fillcolor=self.INVALID_COLOR, style='filled') # root node
        self.navigate(root_node)
        eval_node = self.exec_tree.node_under_evaluation
        if eval_node is not None:
            self.graph.node(str(eval_node.ev_id), str(eval_node.get_name()), fillcolor=self.NODE_IN_EVALUATION, style='filled')
        buggy_node = self.exec_tree.buggy_node
        if buggy_node is not None:
            self.graph.node(str(buggy_node.ev_id), str(buggy_node.get_name()), fillcolor=self.BUGGY_NODE_COLOR, style='filled')
        if self.exec_tree.dependencies is not None:
            for d in self.exec_tree.dependencies: # this loop draws the provenance links between nodes
                self.graph.edge(str(d.source.ev_id), str(d.target.ev_id), None, color=self.PROVENANCE_EDGE_COLOR, dir='forward')
                ## BEGIN customization
                self.graph.node(str(d.source.ev_id), None, fillcolor=self.PROVENANCE_NODE_COLOR, style='filled')
                self.graph.node(str(d.target.ev_id), None, fillcolor=self.PROVENANCE_NODE_COLOR, style='filled')
                ## END customization
    
    

In [30]:
vis = CustomVisualization(prov.exec_tree)

In [33]:
vis.view_exec_tree('exec_tree_p1')

In [32]:
######################################################################################################################